In [1]:
import nilearn
from nilearn import image
import tensorflow as tf
#from keras.applications.nasnet import preprocess_input
#from keras.applications.nasnet import NASNetLarge
#from keras.applications.nasnet import decode_predictions
#from keras.preprocessing.image import load_img
#from keras.preprocessing.image import img_to_array
import numpy as np
import pandas as pd
import nibabel as nib
import json, pickle
import os
import matplotlib.pyplot as plt

import nibabel as nib
from nilearn.image import mean_img

Matplotlib created a temporary config/cache directory at /tmp/pbs.4029679.pbsha.ib.sockeye/matplotlib-_hdi47zv because the default path (/home/mkhademi/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories
Fontconfig error: No writable cache directories


In [2]:
data_dir = '/arc/project/st-ipor-1/mkhademi/BOLD5000_2.0/'
output_dir = '/scratch/st-ipor-1/mkhademi/'
# roi_dir = data_dir + 'BOLD5000_GLMsingle_ROI_betas/py/'
roi_dir = output_dir + 'image_data/parcels/'
warp_dir = '/scratch/st-ipor-1/mkhademi/image_data/MRICroGL_extractedBrain/'
with open(data_dir + 'image_data/MSCOCO/annotations/' + 'instances_train2014.json') as json_data:
    coco_anns = json.load(json_data)
    json_data.close()

In [3]:
imagenet_anns = pd.read_csv(data_dir + 'image_data/LOC_train_solution.csv', sep = ',')
f = open(data_dir + 'image_data/LOC_synset_mapping.txt', 'r')
imagenet_categories = []
for x in f:
    imagenet_categories.append(x.split()[0])
f.close()

In [4]:
with open(data_dir + 'image_data/extra_annotations.pickle', 'rb') as f:
    extra_annotations = pickle.load(f)

In [5]:
cat_convert = {0: [], 1: [444], 2: [705, 751, 817, 829], 3: [], 4: [403], 5: [779, 654], 6: [], 7: [555, 569, 717, 864], 
               8: [449, 554, 625, 833, 814], 9: [920], 10: [], 11: [919], 12: [], 13: [704], 14: [703], 
               15: [134] + [i for i in range(7, 25)], 16: [i for i in range(281, 286)], 17: [i for i in range(151, 269)], 
               18: [], 19: [349], 20: [], 21: [385, 386], 22: [388], 23: [340], 24: [], 25: [515], 26: [414], 27: [879], 
               28: [770], 29: [], 30: [], 31: [457], 32: [], 33: [], 34: [795], 35: [], 36: [805, 852, 522, 574, 722, 768],
               37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [898, 907, 440], 44: [923], 45: [], 46: [968], 47: [],
               48: [623], 49: [910], 50: [659, 809], 51: [954], 52: [], 53: [], 54: [950], 55: [937], 56: [], 57: [934],
               58: [963], 59: [], 60: [], 61: [423, 559, 765], 62: [831], 63: [], 64: [], 65: [], 66: [532], 67: [904],
               68: [526], 69: [861], 70: [799], 71: [851], 72: [620], 73: [673], 74: [761], 75: [878], 76: [487], 77: [651],
               78: [], 79: [859], 80: [], 81: [760], 82: [], 83: [917], 84: [409, 530, 892], 85: [883], 86: [], 87: [850],
               88: [589], 89: [], 90: []}
cat_conv_rev = {}
for key, val in cat_convert.items():
    for x in val:
        cat_conv_rev[x] = key
super_cat = {0: [0], 1: [i for i in range(1, 9)], 2: [i for i in range(9, 15)], 3: [i for i in range(15, 25)],
             4: [i for i in range(25, 33)], 5: [i for i in range(33, 43)], 6: [i for i in range(43, 51)],
             7: [i for i in range(51, 61)], 8: [i for i in range(61, 71)], 9: [i for i in range(71, 77)],
             10: [i for i in range(77, 83)], 11: [i for i in range(83, 91)]}
tool = [27, 30, 38, 39, 42, 47, 48, 49, 73, 74, 76, 86, 88, 89] 
super_cat_rev = {}
for key, val in super_cat.items():
    for x in val:
        super_cat_rev[x] = key

In [6]:
subjs = ['CSI1', 'CSI2', 'CSI3', 'CSI4']
sub = subjs[0]  
if sub == 'CSI4':
    seses = ['ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'ses-06', 'ses-07', 'ses-08',
             'ses-09'] 
else:
    seses = ['ses-01', 'ses-02', 'ses-03', 'ses-04', 'ses-05', 'ses-06', 'ses-07', 'ses-08', 
             'ses-09', 'ses-10', 'ses-11', 'ses-12', 'ses-13', 'ses-14', 'ses-15']
'''
if sub == subjs[0]:
    roi_names = ['LHEarlyVis','LHLOC','LHOPA','LHPPA','LHRSC','RHEarlyVis','RHLOC','RHOPA',
                 'RHPPA','RHRSC']
if sub == subjs[1]:
    roi_names = ['LHEarlyVis','LHLO','LHOPA','LHPPA','LHRSC','RHEarlyVis','RHLO','RHOPA',
                 'RHPPA','RHRRSC']
if sub == subjs[2]:
    roi_names = ['LHEarlyVis','LHLO','LHOPA','LHPPA','LHRSC','RHEarlyVis','RHLO','RHOPA',
                 'RHPPA','RHRSC']
if sub == subjs[3]:
    roi_names = ['LHEarlyVis','LHLOC','LHOPA','LHPPA','LHRSC','RHEarlyVis','RHLOC','RHOPA',
                 'RHPPA','RHRSC']
'''
roi_names=['lFFA','rFFA','lOFA','rOFA','lSTS','rSTS','lLOC','rLOC','lPPA','rPPA','lRSC','rRSC','lTOS','rTOS','lEBA','rEBA']
imgnames = []
f = open(data_dir + sub + '_imgnames.txt', 'r')
for imgname in f:
    imgnames.append(imgname[:-1])
f.close()

In [7]:
img_dict = {}
img_dict_super = {}
coco_category_count = {}
coco_super_count = {}
coco_category_count_trials = {}
coco_super_count_trials = {}
imagenet_category_count = {}
imagenet_super_count = {}
imagenet_category_count_trials = {}
imagenet_super_count_trials = {}
f_coco = open(output_dir + 'image_data/coco_cat.txt', 'w')
f_imagenet = open(output_dir + 'image_data/imagenet_cat.txt', 'w')
coco_total = 0
imagenet_total = 0
coco_total_trials = 0
imagenet_total_trials = 0

for imgname in imgnames:
    if imgname[0]  == 'C': 
        # os.system('cp ' + data_dir + 'image_data/MSCOCO/images/train2014/' + imgname + ' ' + data_dir + 'image_data/drop_box_coco/')
        img_id = int(imgname[15:27]) 
        
        tmp_set = set()
        super_tmp_set = set() 
        coco_total_trials += 1 
        if extra_annotations[imgname] == 'face':
            if 's_0' in coco_super_count_trials:
                coco_super_count_trials['s_0'] += 1
            else:
                coco_super_count_trials['s_0'] = 1
        for i in range(len(coco_anns['annotations'])):
            if coco_anns['annotations'][i]['image_id'] == img_id:
                category_id = coco_anns['annotations'][i]['category_id'] - 1
                super_id = super_cat_rev[category_id]
                if not 'c_' + str(category_id) in tmp_set:
                    if 'c_' + str(category_id) in coco_category_count_trials:
                        coco_category_count_trials['c_' + str(category_id)] += 1
                    else:
                        coco_category_count_trials['c_' + str(category_id)] = 1
                    tmp_set.add('c_' + str(category_id))
                if super_id > 0:    
                    if not 's_' + str(super_id) in super_tmp_set:
                        if 's_' + str(super_id) in coco_super_count_trials:
                            coco_super_count_trials['s_' + str(super_id)] += 1
                        else:
                            coco_super_count_trials['s_' + str(super_id)] = 1
                        super_tmp_set.add('s_' + str(super_id)) 
                    
        tmp_set = set()
        super_tmp_set = set()
        if not imgname in img_dict:
            f_coco.write(imgname +': ')  
            img_dict[imgname] = np.zeros(90, dtype=np.int32)
            img_dict_super[imgname] = np.zeros(12, dtype=np.int32)
            coco_total += 1 
            if extra_annotations[imgname] == 'face':
                img_dict_super[imgname][0] = 1
                if 's_0' in coco_super_count:
                    coco_super_count['s_0'] += 1
                else:
                    coco_super_count['s_0'] = 1 
            for i in range(len(coco_anns['annotations'])):
                if coco_anns['annotations'][i]['image_id'] == img_id:
                    category_id = coco_anns['annotations'][i]['category_id'] - 1
                    img_dict[imgname][category_id] = 1
                    super_id = super_cat_rev[category_id]
                    if not 'c_' + str(category_id) in tmp_set:
                        if 'c_' + str(category_id) in coco_category_count:
                            coco_category_count['c_' + str(category_id)] += 1
                        else:
                            coco_category_count['c_' + str(category_id)] = 1
                        tmp_set.add('c_' + str(category_id))
                    if super_id > 0:   
                        if not 's_' + str(super_id) in super_tmp_set:
                            img_dict_super[imgname][super_id] = 1 
                            if 's_' + str(super_id) in coco_super_count:
                                coco_super_count['s_' + str(super_id)] += 1
                            else:
                                coco_super_count['s_' + str(super_id)] = 1
                            super_tmp_set.add('s_' + str(super_id))    
                    #if category_id == 0:
                    # # # os.system('cp ' + data_dir + 'image_data/MSCOCO/images/train2014/' + imgname + ' ' + data_dir + 'image_data/person_coco/')
                    f_coco.write('c_' + str(category_id) + ' ')
            f_coco.write('\n')
    if imgname[0]  == 'n' and (imgname[1] == '0' or imgname[1] == '1'):
        # os.system('cp ' + data_dir + 'image_data/ILSVRC/Data/CLS-LOC/train/' + imgname[0:9] + '/' + imgname + ' ' + data_dir + 'image_data/drop_box_imagenet/')
        category_id = imagenet_categories.index(imgname[:9])
        flag = False
        if extra_annotations[imgname] == 'person_noface' or extra_annotations[imgname] == 'face':
            imagenet_total_trials += 1
            flag = True
            if 'c_0' in imagenet_category_count_trials:
                imagenet_category_count_trials['c_0'] += 1
            else:
                imagenet_category_count_trials['c_0'] = 1
        if extra_annotations[imgname] == 'face':
            if 's_0' in imagenet_super_count_trials:
                imagenet_super_count_trials['s_0'] += 1
            else:
                imagenet_super_count_trials['s_0'] = 1
        
        if category_id in cat_conv_rev:
            if not flag:
                imagenet_total_trials += 1
            
            super_id = super_cat_rev[cat_conv_rev[category_id]]
            if 'c_' + str(cat_conv_rev[category_id]) in imagenet_category_count_trials:
                imagenet_category_count_trials['c_' + str(cat_conv_rev[category_id])] += 1
            else:
                imagenet_category_count_trials['c_' + str(cat_conv_rev[category_id])] = 1

            if 's_' + str(super_id) in imagenet_super_count_trials:
                imagenet_super_count_trials['s_' + str(super_id)] += 1
            else:
                imagenet_super_count_trials['s_' + str(super_id)] = 1
        
        if not imgname in img_dict:
            img_dict_super[imgname] = np.zeros(12, dtype=np.int32)
            flag = False
            if extra_annotations[imgname] == 'person_noface' or extra_annotations[imgname] == 'face':
                imagenet_total += 1
                flag = True
                if 'c_0' in imagenet_category_count:
                    imagenet_category_count['c_0'] += 1
                else:
                    imagenet_category_count['c_0'] = 1
            if extra_annotations[imgname] == 'face':
                img_dict_super[imgname][0] = 1
                if 's_0' in imagenet_super_count:
                    imagenet_super_count['s_0'] += 1
                else:
                    imagenet_super_count['s_0'] = 1

            f_imagenet.write(imgname +': ')
            f_imagenet.write('c_' + str(category_id) + ' ')
            f_imagenet.write('\n')
            if category_id in cat_conv_rev:
                if not flag:
                    imagenet_total += 1
                if 'c_' + str(cat_conv_rev[category_id]) in imagenet_category_count:
                    imagenet_category_count['c_' + str(cat_conv_rev[category_id])] += 1
                else:
                    imagenet_category_count['c_' + str(cat_conv_rev[category_id])] = 1
                    
                super_id = super_cat_rev[cat_conv_rev[category_id]] 
                img_dict_super[imgname][super_id] = 1
                if 's_' + str(super_id) in imagenet_super_count:
                    imagenet_super_count['s_' + str(super_id)] += 1
                else:
                    imagenet_super_count['s_' + str(super_id)] = 1
                    
            img_dict[imgname] = np.zeros(1000, dtype=np.int32)
            img_dict[imgname][category_id] = 1

f_coco.close()
f_imagenet.close()

In [8]:
for i in range(91):
    if 'c_'+ str(i) in imagenet_category_count:
        print((i, imagenet_category_count['c_'+ str(i)]), end=', ')
    else: 
        print((i, 0), end=', ')

(0, 297), (1, 2), (2, 8), (3, 0), (4, 2), (5, 4), (6, 0), (7, 8), (8, 10), (9, 2), (10, 0), (11, 2), (12, 0), (13, 2), (14, 2), (15, 38), (16, 10), (17, 236), (18, 0), (19, 2), (20, 0), (21, 4), (22, 2), (23, 2), (24, 0), (25, 2), (26, 2), (27, 2), (28, 2), (29, 0), (30, 0), (31, 2), (32, 0), (33, 0), (34, 2), (35, 0), (36, 12), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 6), (44, 2), (45, 0), (46, 2), (47, 0), (48, 2), (49, 2), (50, 4), (51, 2), (52, 0), (53, 0), (54, 2), (55, 2), (56, 0), (57, 2), (58, 2), (59, 0), (60, 0), (61, 6), (62, 2), (63, 0), (64, 0), (65, 0), (66, 2), (67, 2), (68, 2), (69, 2), (70, 2), (71, 2), (72, 2), (73, 2), (74, 2), (75, 2), (76, 2), (77, 2), (78, 0), (79, 2), (80, 0), (81, 2), (82, 0), (83, 2), (84, 6), (85, 2), (86, 0), (87, 2), (88, 2), (89, 0), (90, 0), 

In [9]:
for i in range(91):
    if 'c_'+ str(i) in imagenet_category_count_trials:
        print((i, imagenet_category_count_trials['c_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ') 

(0, 324), (1, 5), (2, 11), (3, 0), (4, 2), (5, 4), (6, 0), (7, 11), (8, 10), (9, 2), (10, 0), (11, 2), (12, 0), (13, 2), (14, 2), (15, 38), (16, 10), (17, 242), (18, 0), (19, 2), (20, 0), (21, 4), (22, 2), (23, 2), (24, 0), (25, 2), (26, 2), (27, 2), (28, 2), (29, 0), (30, 0), (31, 2), (32, 0), (33, 0), (34, 2), (35, 0), (36, 12), (37, 0), (38, 0), (39, 0), (40, 0), (41, 0), (42, 0), (43, 9), (44, 2), (45, 0), (46, 2), (47, 0), (48, 2), (49, 2), (50, 4), (51, 2), (52, 0), (53, 0), (54, 2), (55, 2), (56, 0), (57, 2), (58, 2), (59, 0), (60, 0), (61, 6), (62, 2), (63, 0), (64, 0), (65, 0), (66, 5), (67, 2), (68, 2), (69, 2), (70, 2), (71, 2), (72, 2), (73, 2), (74, 2), (75, 2), (76, 2), (77, 2), (78, 0), (79, 2), (80, 0), (81, 2), (82, 0), (83, 2), (84, 6), (85, 5), (86, 0), (87, 2), (88, 2), (89, 0), (90, 0), 

In [10]:
for i in range(12):
    if 's_'+ str(i) in imagenet_super_count:
        print((i, imagenet_super_count['s_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 215), (1, 34), (2, 8), (3, 294), (4, 10), (5, 14), (6, 18), (7, 10), (8, 18), (9, 12), (10, 6), (11, 14), 

In [11]:
for i in range(12):
    if 's_'+ str(i) in imagenet_super_count_trials:
        print((i, imagenet_super_count_trials['s_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 233), (1, 43), (2, 8), (3, 300), (4, 10), (5, 14), (6, 21), (7, 10), (8, 21), (9, 12), (10, 6), (11, 17), 

In [12]:
for i in range(91):
    if 'c_'+ str(i) in coco_category_count:
        print((i, coco_category_count['c_'+ str(i)]), end=', ')
    else: 
        print((i, 0), end=', ')

(0, 916), (1, 39), (2, 156), (3, 41), (4, 63), (5, 46), (6, 62), (7, 65), (8, 48), (9, 64), (10, 36), (11, 0), (12, 19), (13, 6), (14, 91), (15, 78), (16, 57), (17, 63), (18, 50), (19, 36), (20, 42), (21, 57), (22, 24), (23, 65), (24, 84), (25, 0), (26, 68), (27, 51), (28, 0), (29, 0), (30, 97), (31, 53), (32, 21), (33, 25), (34, 79), (35, 43), (36, 57), (37, 30), (38, 43), (39, 55), (40, 38), (41, 82), (42, 46), (43, 104), (44, 0), (45, 36), (46, 129), (47, 40), (48, 55), (49, 52), (50, 118), (51, 42), (52, 46), (53, 28), (54, 48), (55, 41), (56, 34), (57, 17), (58, 37), (59, 27), (60, 26), (61, 177), (62, 67), (63, 71), (64, 49), (65, 0), (66, 175), (67, 0), (68, 0), (69, 37), (70, 0), (71, 63), (72, 55), (73, 40), (74, 42), (75, 38), (76, 70), (77, 22), (78, 46), (79, 4), (80, 87), (81, 38), (82, 0), (83, 87), (84, 94), (85, 64), (86, 13), (87, 22), (88, 4), (89, 10), (90, 0), 

In [13]:
for i in range(91):
    if 'c_'+ str(i) in coco_category_count:
        print((i, coco_category_count['c_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 916), (1, 39), (2, 156), (3, 41), (4, 63), (5, 46), (6, 62), (7, 65), (8, 48), (9, 64), (10, 36), (11, 0), (12, 19), (13, 6), (14, 91), (15, 78), (16, 57), (17, 63), (18, 50), (19, 36), (20, 42), (21, 57), (22, 24), (23, 65), (24, 84), (25, 0), (26, 68), (27, 51), (28, 0), (29, 0), (30, 97), (31, 53), (32, 21), (33, 25), (34, 79), (35, 43), (36, 57), (37, 30), (38, 43), (39, 55), (40, 38), (41, 82), (42, 46), (43, 104), (44, 0), (45, 36), (46, 129), (47, 40), (48, 55), (49, 52), (50, 118), (51, 42), (52, 46), (53, 28), (54, 48), (55, 41), (56, 34), (57, 17), (58, 37), (59, 27), (60, 26), (61, 177), (62, 67), (63, 71), (64, 49), (65, 0), (66, 175), (67, 0), (68, 0), (69, 37), (70, 0), (71, 63), (72, 55), (73, 40), (74, 42), (75, 38), (76, 70), (77, 22), (78, 46), (79, 4), (80, 87), (81, 38), (82, 0), (83, 87), (84, 94), (85, 64), (86, 13), (87, 22), (88, 4), (89, 10), (90, 0), 

In [14]:
for i in range(91):
    if 'c_'+ str(i) in coco_category_count_trials:
        print((i, coco_category_count_trials['c_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 973), (1, 42), (2, 168), (3, 47), (4, 69), (5, 52), (6, 71), (7, 74), (8, 51), (9, 70), (10, 39), (11, 0), (12, 22), (13, 9), (14, 97), (15, 81), (16, 63), (17, 66), (18, 53), (19, 39), (20, 45), (21, 66), (22, 27), (23, 65), (24, 87), (25, 0), (26, 71), (27, 51), (28, 0), (29, 0), (30, 100), (31, 53), (32, 21), (33, 28), (34, 82), (35, 43), (36, 57), (37, 36), (38, 46), (39, 58), (40, 38), (41, 88), (42, 46), (43, 119), (44, 0), (45, 45), (46, 138), (47, 46), (48, 64), (49, 61), (50, 127), (51, 42), (52, 46), (53, 37), (54, 57), (55, 41), (56, 34), (57, 20), (58, 40), (59, 33), (60, 26), (61, 183), (62, 67), (63, 74), (64, 49), (65, 0), (66, 193), (67, 0), (68, 0), (69, 40), (70, 0), (71, 63), (72, 58), (73, 43), (74, 42), (75, 38), (76, 70), (77, 22), (78, 46), (79, 4), (80, 93), (81, 38), (82, 0), (83, 90), (84, 100), (85, 67), (86, 13), (87, 22), (88, 4), (89, 10), (90, 0), 

In [15]:
for i in range(12):
    if 's_'+ str(i) in coco_super_count:
        print((i, coco_super_count['s_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 536), (1, 400), (2, 195), (3, 527), (4, 228), (5, 396), (6, 291), (7, 250), (8, 390), (9, 178), (10, 120), (11, 265), 

In [16]:
for i in range(12):
    if 's_'+ str(i) in coco_super_count_trials:
        print((i, coco_super_count_trials['s_'+ str(i)]), end=', ')
    else:
        print((i, 0), end=', ')

(0, 560), (1, 439), (2, 210), (3, 563), (4, 234), (5, 417), (6, 321), (7, 274), (8, 414), (9, 181), (10, 126), (11, 277), 

In [17]:
'''
for roi_name in roi_names:
    file_name = roi_dir+sub + '_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_allses_' + roi_name + '.npy'
    roi = np.load(file_name)
    pad = np.zeros((roi.shape[0],522-roi.shape[1]), dtype=np.float32)
    roi = np.concatenate((roi, pad), axis=1) 
    if roi_name == roi_names[0]:
        rois = roi
    else:
        rois = np.concatenate((rois, roi), axis=1)
'''    
# nasnet = tf.keras.applications.nasnet.NASNetLarge()
# tf.keras.models.save_model(nasnet, '/scratch/st-ipor-1/mkhademi/image_data/NASNetlarge.h5')
nasnet = tf.keras.models.load_model('/scratch/st-ipor-1/mkhademi/image_data/NASNetlarge.h5')
filename = output_dir + 'image_data/' + sub + 'bold5000_coco.tfrecords'
writer_coco = tf.io.TFRecordWriter(filename)
filename = output_dir + 'image_data/' + sub + 'bold5000_common.tfrecords'
writer_common = tf.io.TFRecordWriter(filename) 
filename = output_dir + 'image_data/' + sub + 'bold5000_common_vehicle.tfrecords'
writer_common_vehicle = tf.io.TFRecordWriter(filename)
filename = output_dir + 'image_data/' + sub + 'bold5000_common_animal.tfrecords'
writer_common_animal = tf.io.TFRecordWriter(filename) 
filename = output_dir + 'image_data/' + sub + 'bold5000_common_food.tfrecords'
writer_common_food = tf.io.TFRecordWriter(filename)
filename = output_dir + 'image_data/' + sub + 'bold5000_common_tool.tfrecords'
writer_common_tool = tf.io.TFRecordWriter(filename)
i = 0
for ses in seses:
    filename = output_dir + 'image_data/' + sub + 'bold5000_common_' + ses + '.tfrecords'
    writer_common_ses = tf.io.TFRecordWriter(filename)
    #img4d = nib.load(data_dir + sub + '_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_' + ses + '.nii.gz')
    img4d = nib.load(warp_dir + sub + '_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_' + ses + '-n' + '.nii.gz')
    # avg = mean_img(img4d)
    # nib.save(avg, output_dir + 'image_data/' + sub + '-' + ses + '-avg.nii.gz')
    selected_volumes = nilearn.image.index_img(img4d, slice(0, img4d.shape[3]))
    for x in nilearn.image.iter_img(selected_volumes):
        '''
        if sub != subjs[0]:
            #x = nilearn.image.resample_img(x, target_shape=(71, 89, 72), target_affine=x.affine)
            x = nilearn.image.resample_img(x, target_shape=(91, 109, 91), target_affine=x.affine)
        '''
        # x_roi = np.copy(rois[i])
        x_resample = nilearn.image.resample_img(x, target_shape=(79, 95, 69), target_affine=x.affine)
        x_resample = np.array(x_resample.dataobj)
        x_resample = np.nan_to_num(x_resample, nan = 0.0)
        for roi_name in roi_names:
            roi = nib.load(roi_dir + roi_name + '.img')
            roi = np.array(roi.dataobj)
            roi = x_resample[np.nonzero(roi)]
            pad = np.zeros((5085-roi.shape[0]), dtype=np.float32)
            roi = np.concatenate((roi, pad), axis=0) 
            if roi_name == roi_names[0]:
                rois = roi
            else:
                rois = np.concatenate((rois, roi), axis=0)
        x_roi = np.copy(rois)
        x = np.array(x.dataobj)
        x = np.nan_to_num(x, nan = 0.0)
        x = np.reshape(x, (-1))
        imgname = imgnames[i]
        i += 1
        common = False
        coco_label = np.zeros(90, dtype=np.int32)
        imagenet_label = np.zeros(1000, dtype=np.int32) 
        if imgname[0] == 'C':
            img_path =  data_dir + 'image_data/MSCOCO/images/train2014/' + imgname 
            coco_label = img_dict[imgname]
            super_label = img_dict_super[imgname]
            tool_label = 0
            for idx in tool:
                if coco_label[idx] == 1:
                    tool_label = 1
                    break       
            common_label = np.concatenate((coco_label, super_label, np.array([tool_label], dtype=np.int32)), axis=0)
            common = True  
        if  imgname[0]  == 'n' and (imgname[1] == '0' or imgname[1] == '1') and imgname in img_dict:
            img_path = data_dir + '/image_data/ILSVRC/Data/CLS-LOC/train/' + imgname[:9] + '/' + imgname
            imagenet_label = img_dict[imgname]
            category_id = imagenet_categories.index(imgname[:9])
            if category_id in cat_conv_rev:
                coco_label[cat_conv_rev[category_id]] = 1
                common = True
            if extra_annotations[imgname] == 'face':
                common = True
                coco_label[0] = 1
            if extra_annotations[imgname] == 'person_noface':
                common = True
                coco_label[0] = 1
            super_label = img_dict_super[imgname]
            tool_label = 0
            for idx in tool:
                if coco_label[idx] == 1:
                    tool_label = 1
                    break 
            common_label = np.concatenate((coco_label, super_label, np.array([tool_label], dtype=np.int32)), axis=0)
        if common:
            image = tf.keras.preprocessing.image.load_img(img_path, target_size=(331, 331))
            image = tf.keras.preprocessing.image.img_to_array(image)
            image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
            image = tf.keras.applications.nasnet.preprocess_input(image)  
            yhat = nasnet.predict(image)
            example = tf.train.Example(features=tf.train.Features(feature={
                'x': tf.train.Feature(float_list=tf.train.FloatList(value=x)),
                'roi': tf.train.Feature(float_list=tf.train.FloatList(value=x_roi)),
                'yhat': tf.train.Feature(float_list=tf.train.FloatList(value=yhat[0])),
                'y_coco': tf.train.Feature(int64_list=tf.train.Int64List(value=coco_label)),
                'y_imagenet': tf.train.Feature(int64_list=tf.train.Int64List(value=imagenet_label)),
                'y_super': tf.train.Feature(int64_list=tf.train.Int64List(value=super_label)),
                'y_common': tf.train.Feature(int64_list=tf.train.Int64List(value=common_label))
                }))
            writer_common.write(example.SerializeToString())
            writer_common_ses.write(example.SerializeToString())
            if imgname[0] == 'C':
                writer_coco.write(example.SerializeToString())
            if common_label[2] or common_label[4] or common_label[6] or common_label[7]:
                writer_common_vehicle.write(example.SerializeToString())
            if common_label[15] or common_label[16] or common_label[17] or common_label[24]:
                writer_common_animal.write(example.SerializeToString())
            if common_label[51] or common_label[52] or common_label[54] or common_label[55]:
                writer_common_food.write(example.SerializeToString())
            if common_label[30] or common_label[48] or common_label[49] or common_label[76]:
                writer_common_tool.write(example.SerializeToString())
    writer_common_ses.close()

writer_coco.close()
writer_common.close()
writer_common_vehicle.close()
writer_common_animal.close()
writer_common_food.close()
writer_common_tool.close()
# -4218170.5

2022-08-24 20:10:57.040963: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-08-24 20:10:57.107076: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-08-24 20:10:57.112122: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-08-24 20:10:57.121910: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-08-24 20:10:57.128274: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-08-24 20:10:57.132148: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so

2022-08-24 20:11:23.382809: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-08-24 20:11:23.590948: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7


In [18]:
print(tf.test.is_gpu_available())
print('done.')
'''
filename = output_dir + 'image_data/' + sub + 'bold5000_recons.tfrecords'
writer_recons = tf.io.TFRecordWriter(filename)
img_rows, img_cols, channels = 56, 56, 3
i=0
imagenet_str = '/image_data/ILSVRC/Data/CLS-LOC/train/'
for ses in seses:
    img4d = nib.load(data_dir + sub + '_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_' + ses + '.nii.gz')
    selected_volumes = nilearn.image.index_img(img4d, slice(0, img4d.shape[3]))
    for x in nilearn.image.iter_img(selected_volumes):
        if sub != subjs[0]:
            x = nilearn.image.resample_img(x, target_shape=(71, 89, 72), target_affine=x.affine)
        x = np.array(x.dataobj)
        x = np.nan_to_num(x, nan = 0.0)
        x = np.reshape(x, (-1))
        imgname = imgnames[i]
        i += 1
        if imgname[0] == 'C':
            img_path =  data_dir + 'image_data/MSCOCO/images/train2014/' + imgname 
        if  imgname[0]  == 'n' and (imgname[1] == '0' or imgname[1] == '1'):
            img_path = data_dir + imagenet_str + imgname[:9] + '/' + imgname
        image = tf.keras.preprocessing.image.load_img(img_path, target_size=(331, 331))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.reshape(tf.cast(tf.image.resize(image, (int(img_rows), int(img_cols))),
                              tf.float32) / 127.5 - 1, (1, img_rows, img_cols, channels))
        image = np.reshape(image,(-1))
        example = tf.train.Example(features=tf.train.Features(feature={
            'x': tf.train.Feature(float_list=tf.train.FloatList(value=x)),
            'image': tf.train.Feature(float_list=tf.train.FloatList(value=image))}))
        writer_recons.write(example.SerializeToString())
writer_recons.close()
print(i)
'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True
done.


2022-08-24 20:37:30.902702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:1a:00.0 name: Tesla V100-SXM2-16GB computeCapability: 7.0
coreClock: 1.53GHz coreCount: 80 deviceMemorySize: 15.78GiB deviceMemoryBandwidth: 836.37GiB/s
2022-08-24 20:37:30.902878: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2022-08-24 20:37:30.902909: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-08-24 20:37:30.902938: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2022-08-24 20:37:30.902966: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcurand.so.10
2022-08-24 20:37:30.902991: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcusolv

"\nfilename = output_dir + 'image_data/' + sub + 'bold5000_recons.tfrecords'\nwriter_recons = tf.io.TFRecordWriter(filename)\nimg_rows, img_cols, channels = 56, 56, 3\ni=0\nimagenet_str = '/image_data/ILSVRC/Data/CLS-LOC/train/'\nfor ses in seses:\n    img4d = nib.load(data_dir + sub + '_GLMbetas-TYPED-FITHRF-GLMDENOISE-RR_' + ses + '.nii.gz')\n    selected_volumes = nilearn.image.index_img(img4d, slice(0, img4d.shape[3]))\n    for x in nilearn.image.iter_img(selected_volumes):\n        if sub != subjs[0]:\n            x = nilearn.image.resample_img(x, target_shape=(71, 89, 72), target_affine=x.affine)\n        x = np.array(x.dataobj)\n        x = np.nan_to_num(x, nan = 0.0)\n        x = np.reshape(x, (-1))\n        imgname = imgnames[i]\n        i += 1\n        if imgname[0] == 'C':\n            img_path =  data_dir + 'image_data/MSCOCO/images/train2014/' + imgname \n        if  imgname[0]  == 'n' and (imgname[1] == '0' or imgname[1] == '1'):\n            img_path = data_dir + imagene